# **기사 전처리(ngramize까지)**

In [1]:
# import
import re
import csv
import numpy as np
import pandas as pd
import MeCab
from ekonlpy.sentiment import MPCK
from ekonlpy.tag import Mecab
from ekonlpy.tag import Mecab

In [2]:
# 기간별 기사가 저장된 CSV파일들을 불러옴
arti_df1 = pd.read_csv('article_2005_20110706.csv')
arti_df2 = pd.read_csv('article_20110707_20160202.csv')
arti_df3 = pd.read_csv('article_20160203_2017.csv')

## **기간별 기사들을 하나의 DataFrame으로 만듬.**

In [3]:
# 각 기사들 중 연합뉴스의 기사들을 각 변수에 저장
yonhap1 = arti_df1[arti_df1['logo'] == '연합뉴스']
yonhap2 = arti_df2[arti_df2['logo'] == '연합뉴스']
yonhap3 = arti_df3[arti_df3['logo'] == '연합뉴스']

In [4]:
# 기간별 기사들을 합친 후 인덱스 재설정
frames = [yonhap1, yonhap2, yonhap3]
yonhap_df = pd.concat(frames)
yonhap_df.reset_index(drop=True, inplace=True)
yonhap_df

,date,logo,text,title,url
0,2005.01.01. 오전 7:01,연합뉴스,(단위:%) ...,<표> 은행 금전신탁배당률.정기예금 금리(12.31일),https://news.naver.com/main/read.nhn?mode=LSD&...
1,2005.01.17. 오후 12:02,연합뉴스,농민 자녀 장학기금 2012년까지 1천억원으로 확대\r\n (서울=연합뉴스) ...,농협 상호금융 대출금리 연 5%대 초반으로 인하,https://news.naver.com/main/read.nhn?mode=LSD&...
2,2005.01.12. 오전 6:06,연합뉴스,(서울=연합뉴스) 김문성기자= 증권사들이 개인 투자자를 증시로 끌어들이기 위해 위...,`개인 투자자 잡아라'..증거금.대출금리 인하,https://news.naver.com/main/read.nhn?mode=LSD&...
3,2005.01.12. 오전 11:45,연합뉴스,(서울=연합뉴스) 신호경기자= 씨티그룹글로벌마켓증권(이하 씨티그룹증권)은 오는 13...,"""이달 금리동결 전망..2월 인하 가능성""<씨티그룹증권>",https://news.naver.com/main/read.nhn?mode=LSD&...
4,2005.01.12. 오후 3:45,연합뉴스,(서울=연합뉴스) 신호경기자= 모건스탠리는 12일 올해 중국이 점진적 금리 인상에 ...,"""中, 올해 점진적 금리인상이 최선""<모건스탠리>",https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
70163,2017.12.30. 오전 11:05,연합뉴스,(상하이=연합뉴스) 정주호 특파원 = 올해 들어 중국 위안화 가치가 5.8% 오르며...,中 위안화 가치 1년새 5.8% 상승…9년만에 최대폭,https://news.naver.com/main/read.nhn?mode=LSD&...
70164,2017.12.30. 오전 8:03,연합뉴스,"국내 주식형, 패시브펀드가 액티브펀드 상회 (서울=연합뉴스) 윤선희 기자 = 전 세...",중국·베트남펀드 올해 수익률 30%대로 최고,https://news.naver.com/main/read.nhn?mode=LSD&...
70165,2017.12.29. 오전 10:05,연합뉴스,(서울=연합뉴스) 이춘규 기자 = 미국과 유럽이 세계 금융위기 뒤 지속한 금융완화에...,"""마이너스금리 일본은행서 드디어 금리인상 검토 목소리""",https://news.naver.com/main/read.nhn?mode=LSD&...
70166,2017.12.29. 오후 12:00,연합뉴스,"고객 몰래 대출금리 인상 혐의…법원 ""무단으로 인상했다 단정 못해""(서울=연합뉴스)...",'대출금리 조작 사건' 외환은행 전·현직 임원 무죄 확정,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
# 각 기사들 중 이데일리의 기사들을 각 변수에 저장
eday1 = arti_df1[arti_df1['logo'] == '이데일리']
eday2 = arti_df2[arti_df2['logo'] == '이데일리']
eday3 = arti_df3[arti_df3['logo'] == '이데일리']

In [6]:
# 기간별 기사들을 합친 후 인덱스 재설정
eday_frames = [eday1, eday2, eday3]
eday_df = pd.concat(eday_frames)
eday_df.reset_index(drop=True, inplace=True)
eday_df

,date,logo,text,title,url
0,2005.01.12. 오후 12:01,이데일리,[edaily 박기수기자] 작년 한해동안 주택금융공사의 모기지론(장기주택담보대출)을...,"작년 고정금리 모기지론 4만7천여명, 3.3조원 대출받아",https://news.naver.com/main/read.nhn?mode=LSD&...
1,2005.01.12. 오후 1:49,이데일리,[edaily 이정훈기자] 최근 장기금리를 중심으로 한 시중금리 상승에 대해 KB자...,"KB자산운용 백경호 사장 ""現 금리상승 일시적""",https://news.naver.com/main/read.nhn?mode=LSD&...
2,2005.01.18. 오후 10:26,이데일리,"[뉴욕=edaily 정명수특파원] 안소니 산토메로 필라델피아 연방은행 총재는 ""올해...","""금리인상 점진적""-필라델피아FRB총재",https://news.naver.com/main/read.nhn?mode=LSD&...
3,2005.01.01. 오전 7:30,이데일리,"[뉴욕=edaily 안근모특파원] 2004년 마지막 거래일의 부진이 아쉽긴 하지만,...",(월가시각)내일에는 내일의 태양이,https://news.naver.com/main/read.nhn?mode=LSD&...
4,2005.01.12. 오전 8:38,이데일리,[edaily 강종구기자]시티글로벌마켓(CGM)은 12일 한국은행의 콜금리 인하에 ...,금리인하는 1월 아닌 2월-시티(1보),https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
99858,2017.12.30. 오전 8:01,이데일리,[이데일리 권소현 기자] ‘부익부 빈인빈’내년 주택 시장은 전반적으로 약보합을 이어...,[2018 부동산 전망]①규제에 입주폭탄까지…보합 속 양극화,https://news.naver.com/main/read.nhn?mode=LSD&...
99859,2017.12.29. 오후 3:59,이데일리,[이데일리 김정현 기자] 가계부채 비중이 높은 우리나라에서 금리정책은 경기부양보다는...,"""韓, 가계부채 높아 금리 인상시 소비에 부정적""",https://news.naver.com/main/read.nhn?mode=LSD&...
99860,2017.12.30. 오전 6:02,이데일리,[이데일리 박종오 유현욱 기자] 올해 금융권 주요 이슈를 한 단어로 설명할 수 있는...,인터넷뱅크·비트코인 열풍…금융권 올해 키워드는 ‘급변’,https://news.naver.com/main/read.nhn?mode=LSD&...
99861,2017.12.30. 오전 8:01,이데일리,[이데일리 권소현 기자] 내년 부동산 시장에 관망세가 짙을 것으로 예상되는 가운데 ...,[2018 부동산 전망]③양도세 중과 직전이 내집마련 적기,https://news.naver.com/main/read.nhn?mode=LSD&...


In [7]:
# 각 기사들 중 연합인포맥스의 기사들을 각 변수에 저장
yonhap_in1 = arti_df1[arti_df1['logo'] == '연합인포맥스']
yonhap_in2 = arti_df2[arti_df2['logo'] == '연합인포맥스']
yonhap_in3 = arti_df3[arti_df3['logo'] == '연합인포맥스']

In [8]:
# 기간별 기사들을 합친 후 인덱스 재설정
yonhap_in_frames = [yonhap_in1, yonhap_in2, yonhap_in3]
yonhap_in_df = pd.concat(yonhap_in_frames)
yonhap_in_df.reset_index(drop=True, inplace=True)
yonhap_in_df

,date,logo,text,title,url
0,2005.01.18. 오후 2:12,연합인포맥스,(서울=연합인포맥스) 이현중기자= 18일 오후 채권시장은 심리회복에 따른 매수세가...,"채권금리, 심리회복..본격 매수세 회복은 아직",https://news.naver.com/main/read.nhn?mode=LSD&...
1,2005.01.12. 오후 10:54,연합인포맥스,(뉴욕=연합인포맥스) 김홍규 특파원= 캐서린 미네한 보스턴연방준비은행 총재는12일 ...,FRB 기준금리 추가 인상 필요[보스턴연銀 총재],https://news.naver.com/main/read.nhn?mode=LSD&...
2,2005.01.12. 오전 10:15,연합인포맥스,"(서울=연합인포맥스) 이규창기자= 향후 新BIS 협약인 바젤Ⅱ가 발효되면, 우리나...","바젤Ⅱ 발효시, 외평채 가산금리 상승",https://news.naver.com/main/read.nhn?mode=LSD&...
3,2005.01.18. 오전 10:44,연합인포맥스,(서울=연합인포맥스) 이한용 기자= 18일 아시아에서 스크린을 통해 거래되는 미국채...,"美 국채가, FRB 금리인상 시사 관측 亞서 하락 전망",https://news.naver.com/main/read.nhn?mode=LSD&...
4,2005.01.18. 오전 9:16,연합인포맥스,(서울=연합인포맥스) 이현중기자= 18일 오전 채권시장은 올들어 이어진 금리 급등흐...,"채권금리, 내림세 출발..심리 안정 기대",https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...
80917,2013.01.02. 오전 8:51,연합인포맥스,(서울=연합인포맥스) 권용욱 기자 = 2일 국채선물시장은 미국 재정절벽 타결 협상에...,[국채선물 전략] 미국發 약세 압력,https://news.naver.com/main/read.nhn?mode=LSD&...
80918,2013.01.02. 오전 8:33,연합인포맥스,"(서울=연합인포맥스) 정지서 기자 = 박재식 한국증권금융 사장은 2일 ""실적 악화가...","박재식 한국증금 사장 ""실적악화 불가피, 중장기 로드맵 필수""",https://news.naver.com/main/read.nhn?mode=LSD&...
80919,2013.01.02. 오전 9:00,연합인포맥스,(서울=연합인포맥스) 권용욱 기자 = 2일 단기자금시장은 통안채 만기 등으로 지급준...,[단기자금시장 분석] 통안채 만기..지준 잉여,https://news.naver.com/main/read.nhn?mode=LSD&...
80920,2013.01.02. 오전 8:06,연합인포맥스,(서울=연합인포맥스) 2일 서울채권시장은 미국의 재정절벽 협상이 연말 극적으로타결된...,[한창헌의 채권분석] 재정절벽 타결에 되돌림 압력,https://news.naver.com/main/read.nhn?mode=LSD&...


## **전처리 함수**

In [9]:
def multi_pre_processing(df, doc_name_str, max_iterate) : 
    test_list = [[1,2,3,4,5,6]]  # 데이터 프레임을 만들기 위한 test_list
    df_pre_processing = pd.DataFrame(test_list,columns =['date', 'title', 'content','tokens', 'ngrams', 'callLabel'])
    df_pre_processing = df_pre_processing.drop([0])  # test_list 제거
    
    mecab = Mecab()
    mpck = MPCK()
    
    for i in range(max_iterate+1) :
        print('{}/{}'.format(i, max_iterate))
        
        text = str(df['text'][i])

        # 이데일리 전처리
        text2 = re.sub('\[.*\]', '', text)  # [서울=edaily 배진우 기자] 등 제거
        text3 = re.sub('Copyright.+\(?[a-zA-z0-9]+@[a-zA-z.]+\)?', '', text2)  # Copyright ... 배진우 기자 (bigboom@edaily.co.kr) 제거
        
        # 연합뉴스, 연합인포맥스 전처리
        text4 = re.sub('\(\w*=\w*\)', '', text3)  # (부산=연합뉴스), (서울=연합인포맥스) 등 제거
        text5 = re.sub('[\w\s]+기자\s?=', '', text4)  # 배진우 기자 =, 배진우 기자= 제거
        text6 = re.sub('[\w\s]+특파원\s?=', '', text5)  # 하은혜 특파원 =, 조하담 특파원= 제거
        text7 = re.sub('[a-zA-z0-9]+@[a-zA-z.]+', '', text6)  # goisugn@yna.co.kr 제거
        text8 = re.sub('\(끝\)', '', text7).strip()  # (끝) 제거
        
        date = str(df['date'][i])[:10].replace('.','-')  # 날짜에서 '.'을 '-'로 변경
        title = str(df['title'][i])
        
        tokens = mpck.tokenize(text)
        
        text = str(df['text'][i]).replace('\n', '')  # 줄바꿈 제거
        
        ngrams = mpck.ngramize(tokens)
        
        # ngrams가 없을 경우 저장하지 않음
        if not ngrams:
            continue
        
        df_pre_processing.loc[i] = [date, title, text8, tokens, ngrams, 0]

    df_pre_processing.reset_index(drop=True, inplace=True)  # 인덱스 재 설정 
    df_pre_processing.to_csv('{}.csv'.format(doc_name_str))

## **TEST**

In [10]:
multi_pre_processing(yonhap_df,'yonhap',100)

0/100
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100


In [11]:
multi_pre_processing(eday_df,'eday',100)

0/100
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100


In [12]:
multi_pre_processing(yonhap_in_df,'yonhap_in',100)

0/100
1/100
2/100
3/100
4/100
5/100
6/100
7/100
8/100
9/100
10/100
11/100
12/100
13/100
14/100
15/100
16/100
17/100
18/100
19/100
20/100
21/100
22/100
23/100
24/100
25/100
26/100
27/100
28/100
29/100
30/100
31/100
32/100
33/100
34/100
35/100
36/100
37/100
38/100
39/100
40/100
41/100
42/100
43/100
44/100
45/100
46/100
47/100
48/100
49/100
50/100
51/100
52/100
53/100
54/100
55/100
56/100
57/100
58/100
59/100
60/100
61/100
62/100
63/100
64/100
65/100
66/100
67/100
68/100
69/100
70/100
71/100
72/100
73/100
74/100
75/100
76/100
77/100
78/100
79/100
80/100
81/100
82/100
83/100
84/100
85/100
86/100
87/100
88/100
89/100
90/100
91/100
92/100
93/100
94/100
95/100
96/100
97/100
98/100
99/100
100/100


In [13]:
df_yonhap_test = pd.read_csv('yonhap.csv', index_col=0)
df_yonhap_test

,date,title,content,tokens,ngrams,callLabel
0,2005-01-17,농협 상호금융 대출금리 연 5%대 초반으로 인하,현행 연 6%대 후반인 농협 상호금융 대출금리가 올해안에 연 5%대 초반까지 단계...,"['농민/NNG', '자녀/NNG', '장학기금/NNG', '확대/NNG', '영복...","['농가/NNG;소득/NNG;안정/NNG', '비용/NNG;부담/NNG;덜/VV',...",0
1,2005-01-12,`개인 투자자 잡아라'..증거금.대출금리 인하,증권사들이 개인 투자자를 증시로 끌어들이기 위해 위탁 증거금 적용 비율과 대출 금...,"['기자/NNG', '개인/NNG', '증시/NNG', '끌/VV', '위하/VV'...","['대출/NNG;금리/NNG;인하/NNG', '대출/NNG;금리/NNG;하향/NNG...",0
2,2005-01-12,"""이달 금리동결 전망..2월 인하 가능성""<씨티그룹증권>",씨티그룹글로벌마켓증권(이하 씨티그룹증권)은 오는 13일 열리는 금융통화위원회에서 한...,"['기자/NNG', '증권/NNG', '오/VV', '열/VV', '콜/NNG', ...","['국고채/NNG;물량/NNG;부담/NNG', '채권시장/NNG;불안/NNG', '...",0
3,2005-01-12,"""中, 올해 점진적 금리인상이 최선""<모건스탠리>",모건스탠리는 12일 올해 중국이 점진적 금리 인상에 나서는 한편 통화가치는 현 수준...,"['기자/NNG', '점진적/VAX', '금리/NNG', '인상/NNG', '나서/...","['인플레이션/NNG;위험/NNG;높/VA;금리/NNG;인상/NNG', '금리/NN...",0
4,2005-01-18,"한국타이어, 지난해 사상최대 실적 달성(종합)",한국타이어가 수출 호조와 고부가가치 제품 판매 증가에 힘입어 지난해 사상 최대 실적...,"['기자/NNG', '수출/NNG', '호조/NNG', '고부가가치/NNG', '제...","['영업이익/NNG;순이익/NNG;증가/NNG', '제품/NNG;판매/NNG;증가/...",0
...,...,...,...,...,...,...
80,2005-01-14,<저축銀 부실 어디까지 왔나>,한중상호저축은행이 14일 부채가 자산을 초과하고 국제결제은행(BIS) 기준 자기자본...,"['고준/NNG', '기자/NNG', '한중/NNG', '상호저축은행/NNG', '...","['bis/NNG;자기자본비율/NNG;낮/VA', '대출/NNG;연체율/NNG;급등...",0
81,2005-01-14,<연합인터뷰> 이헌재 경제부총리,"이헌재 부총리 겸 재정경제부 장관은 14일 ""금융기관들은 과거 도덕적 해이로 인한 ...","['기자/NNG', '금융기관/NNG', '도덕적해이/NNG', '인하/VV', '...","['물가/NNG;불안/NNG;우려/NNG', '불안/NNG;우려/NNG;않/VX',...",0
82,2005-01-14,작년 국고채 회전율 858%..환란 이후 최고,손바뀜 회사채의 14배.. 경기침체 등으로 안전성이 높은 투자대상에 대한 선호도가 ...,"['손바뀜/NNG', '회사채/NNG', '배/NNG', '안전성/NNG', '선호...","['국고채/NNG;증가/NNG', '경기/NNG;침체/NNG']",0
83,2005-01-14,미국 달러화 강세,미국의 연방기금금리 인상 속도가 빨라질 수 있다는 관측이 나오면서 미국 달러화가 강...,"['기자/NNG', 'ff/NNG', '금리/NNG', '인상/NNG', '속도/N...","['fed/NNG;금리/NNG;인상/NNG', 'ff/NNG;금리/NNG;인상/NNG']",0


In [14]:
df_eday_test = pd.read_csv('eday.csv', index_col=0)
df_eday_test

,date,title,content,tokens,ngrams,callLabel
0,2005-01-12,"작년 고정금리 모기지론 4만7천여명, 3.3조원 대출받아",작년 한해동안 주택금융공사의 모기지론(장기주택담보대출)을 통해 4만7841명이 3조...,"['기자/NNG', '모기지/NNG', '장기/NNG', '주택담보대출/NNG', ...",['실적/NNG;오르/VV'],0
1,2005-01-12,"KB자산운용 백경호 사장 ""現 금리상승 일시적""","최근 장기금리를 중심으로 한 시중금리 상승에 대해 KB자산운용 백경호 사장은 ""마찰...","['기자/NNG', '장기/NNG', '금리/NNG', '중심/NNG', '하/VV...","['개선/NNG;내수/NNG;회복/NNG', '내수/NNG;회복/NNG;지연/NNG...",0
2,2005-01-18,"""금리인상 점진적""-필라델피아FRB총재","안소니 산토메로 필라델피아 연방은행 총재는 ""올해 경제는 좀 더 탄탄한 성장세를 나...","['뉴욕/NNG', '특파원/NNG', '경제/NNG', '좀/MAG', '더/MA...","['생산성/NNG;증가율/NNG;둔화/NNG', 'fed/NNG;금리/NNG;인상/...",0
3,2005-01-01,(월가시각)내일에는 내일의 태양이,"2004년 마지막 거래일의 부진이 아쉽긴 하지만, 할 만큼은 다 했다. 라이언 벡 ...","['뉴욕/NNG', '안/MAG', '근모/NNG', '특파원/NNG', '마지막/...","['걸림돌/NNG;금리/NNG;인상/NNG', '경제/NNG;성장률/NNG;낮/VA...",0
4,2005-01-12,금리인하는 1월 아닌 2월-시티(1보),시티글로벌마켓(CGM)은 12일 한국은행의 콜금리 인하에 대한 기대가 높지만 그 시...,"['기자/NNG', '시티/NNG', '글로벌/NNG', '시장/NNG', '콜/N...","['콜/NNG;금리/NNG;인하/NNG;기대/NNG;높/VA', '세계/NNG;경제...",0
...,...,...,...,...,...,...
81,2005-01-20,(고령화시대)③채권시장이 변한다,저출산·고령화는 채권시장에도 직간접적인 영향을 미친다. 채권시장은 고령화와 함께 저...,"['기자/NNG', '고령화/NNG', '채권시장/NNG', '직간접적/NNG', ...","['채권시장/NNG;콜/NNG;금리/NNG;인하/NNG', '장기물/NNG;시장/N...",0
82,2005-01-20,(마켓서핑)싹트는 불안감,채권시장은 물량부담에 대한 불안감이 투자심리를 위축시켰고 주식시장에서는 단기 급등에...,"['기자/NNG', '채권시장/NNG', '물량/NNG', '부담/NNG', '불안...","['단기/NNG;급등/NNG;가격/NNG;부담/NNG', '채권시장/NNG;물량/N...",0
83,2005-01-20,"""실적 들쭉날쭉"" 나스닥 2100 무너져",다우와 나스닥이 약보합선에 머물러 있다. 나스닥은 다시 2100선이 무너졌다. 기업...,"['뉴욕/NNG', '특파원/NNG', '다우/NNG', '나스닥/NNG', '약보...","['근원/NNG;소비자/NNG;물가/NNG;상승/NNG', '경제/NNG;지표/NN...",0
84,2005-01-20,"""실적 신통치 않다""..나스닥 1.5%↓",다우와 나스닥이 큰 폭으로 떨어졌다. 기술주 실적에 대한 의구심이 고조되면서 장막판...,"['뉴욕/NNG', '특파원/NNG', '다우/NNG', '나스닥/NNG', '크/...","['국채/NNG;수익률/NNG;소폭/MAG;하락/NNG', '근원/NNG;소비자/N...",0


In [15]:
df_yonhap_in_test = pd.read_csv('yonhap_in.csv', index_col=0)
df_yonhap_in_test

,date,title,content,tokens,ngrams,callLabel
0,2005-01-18,"채권금리, 심리회복..본격 매수세 회복은 아직",18일 오후 채권시장은 심리회복에 따른 매수세가 유입되면서 주요 기물별 금리하락폭...,"['기자/NNG', '채권시장/NNG', '심리/NNG', '회복/NNG', '따르...","['채권시장/NNG;심리/NNG;회복/NNG', '금리/NNG;하락/NNG;크/VV...",0
1,2005-01-12,FRB 기준금리 추가 인상 필요[보스턴연銀 총재],캐서린 미네한 보스턴연방준비은행 총재는12일 연방준비제도이사회(FRB)가 여전히 기...,"['뉴욕/NNG', '특파원/NNG', '보스턴/NNG', 'fed/NNG', 'f...","['경상/NNG;적자/NNG', '완만/NNG;성장/NNG']",0
2,2005-01-12,"바젤Ⅱ 발효시, 외평채 가산금리 상승","향후 新BIS 협약인 바젤Ⅱ가 발효되면, 우리나라의 외국환평형기금채권(외평채)의 ...","['기자/NNG', '新/NNG', 'bis/NNG', '협약/NNG', '바젤/N...","['외평채/NNG;스프레드/NNG;상승/NNG', '채권/NNG;스프레드/NNG;상...",0
3,2005-01-18,"美 국채가, FRB 금리인상 시사 관측 亞서 하락 전망",18일 아시아에서 스크린을 통해 거래되는 미국채 가격은 연방준비제도이사회(FRB) ...,"['기자/NNG', '스크린/NNG', '통하/VV', '거래/NNG', '되/XS...","['인상/NNG;관측/NNG;강화/NNG', '경제/NNG;회복/NNG;가속/NNG...",0
4,2005-01-18,"채권금리, 내림세 출발..심리 안정 기대",18일 오전 채권시장은 올들어 이어진 금리 급등흐름이 주춤하면서 내림세로 출발했다....,"['기자/NNG', '채권시장/NNG', '들/VV', '이어지/VV', '금리/N...","['금리/NNG;급등/NNG', '심리/NNG;안정/NNG', '기대/NNG;살/VV']",0
...,...,...,...,...,...,...
92,2005-01-12,"美달러, `환율 시장에 의해 결정돼야' 발언에 무너져","""미국 정부는 달러강세정책을 유지하고 있으나 환율은 시장에 의해 결정돼야 한다""는 ...","['뉴욕/NNG', '특파원/NNG', '달러/NNG', '강세/NNG', '정책/...","['무역/NNG;경상/NNG;적자/NNG', '무역/NNG;적자/NNG;축소/NNG...",0
93,2005-01-12,"[채권금리 급등, 은행권 운용담당자들 반응]",- 금리급등 비정상적.. 은행권 자금운용 담당자들은 10년물 공급물량으로 촉발된 채...,"['금리/NNG', '급등/NNG', '비/NNG', '정상/NNG', '발행/NN...","['물량/NNG;공급/NNG;확대/NNG', '콜/NNG;금리/NNG;인하/NNG'...",0
94,2005-01-12,"채권금리, 보합권 불안한 횡보",12일 오후 채권시장은 전일의 급등분위기가다소 진정되면서 보합권에서 횡보흐름을 ...,"['기자/NNG', '채권시장/NNG', '급등/NNG', '분위기/NNG', '다...",['채권시장/NNG;급등/NNG'],0
95,2005-01-20,"은행채 발행물량 줄줄이..""금리안정 기다렸다""",시장금리가 다소 안정세를 찾아가면서 은행채 발행물량이 쏟아지고 있다. 20일...,"['황병/NNG', '극/NNG', '기자/NNG', '시장/NNG', '금리/NN...","['금리/NNG;상승/NNG;시장/NNG;불안/NNG', '발행/NNG;여건/NNG...",0


In [16]:
df_yonhap_test['content'][20]

'한은차입금 등으로 재정조기집행 자금충당\r\n1월 예정 10년장기물중 일부는 5년물로 변경\r\n     윤근영. 정부는 올해 69조1천억원의 국채를  발행할 계획이어서 연말에는 국채발행 잔액이 227조9천400억원에 이를 전망이다.\r\n    정부는 또 경기부양에 필요한 자금을 확보하기 위해 한국은행 차입 및 재정증권 발행한도를 작년의 8조원에서 올해는 18조원으로 늘렸다.\r\n    이와 함께 장기채 시장안정과 저금리 기조를 위해 오는 17일 발행키로 했던  10년물 3조1천800억원어치 가운데 1조5천억원어치를 5년물로 바꿔 24일 발행키로 했다.\r\n    재정경제부가 12일 발표한 `올해 국고채 발행계획\'에 따르면 올해 국채 발행 예정액은 국고채 59조4천억원, 외국환평형기금채권 1조2천억원(10억달러), 국민주택채권 8조5천억원 등 모두 69조1천억원이다.\r\n    정부가 올해 국채를 한도내에서 전액 발행하면 올해말 국채발행 잔액은  227조9천억원으로 작년말의 183조3천821억원보다 24.3%나 늘어나게 된다. \r\n    국고채로 조달되는 자금 가운데 45조6천억원은 ▲공적자금의 국채전환  13조6천억원 ▲외환시장 안정을 위한 재원 조달 21조9천억원 ▲일반회계 적자보전 5조7천억원 등 신규 재정수요 충당을 위해 사용되고 나머지 13조8천억원은 기존 국채 상환에 이용된다.\r\n    정부는 또 외화표시 외평채를 발행, 국가 신인도 제고와 한국경제 홍보에  활용하고 국민주택채권 발행으로 조달되는 자금은 기존 국민주택채권 상환(3조8천억원), 주택 건설 및 주택수요자 융자자금(4조7천억원) 등에 사용할 예정이다.\r\n    이철환 재경부 국고국장은 "국채발행 규모와 시기 등은 경기회복을 위한 저금리기조 유지에 초점을 맞출 계획"이라면서 "한은차입 및 재정증권 발행한도를 작년  8조원에서 올해는 18조원으로 확대한 것도 재정조기 집행에 필요한 자금을  확보하되 저금리기조에 부담을 주지 않기 위한 것"이라고 설명했다. \r\n    